In [3]:
import argparse, os
from tqdm import tqdm
import torch
import numpy as np
from transformers import DPTForDepthEstimation
from PIL import Image

In [2]:
gpu = 0
subject = 'subj01'
# roi = ['early', 'ventral', 'midventral', 'midlateral', 'lateral', 'parietal']
roi = 'midventral'

In [5]:
datedir = f'../data/decoded/{subject}/'
savedir = f'../data/decoded/{subject}/dpt_fromemb/'
os.makedirs(f'{savedir}', exist_ok=True)
savedir_img = f'../data/decoded/{subject}/dpt_fromemb_image/'
os.makedirs(f'{savedir_img}', exist_ok=True)

In [6]:
device = torch.device(f"cuda:{gpu}") if torch.cuda.is_available() else torch.device("cpu")
model = DPTForDepthEstimation.from_pretrained("./dpt_large")
model.to(device)
imsize = (512,512)
latentsize = (64,64)

Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at ./dpt_large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution1.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
dpt_embs = [] 
# for idx in range(4):
#     fname = f'{datdir}/{subject}_{"_".join(roi)}_scores_dpt_emb{idx}.npy'
#     dpt_embs.append(np.load(fname))
dpt_embs.append(np.load(f'{datedir}/subj01_midventral_brain_embs_dpt.npy'))
dpt_embs = np.stack(dpt_embs)
dpt_embs = torch.Tensor(dpt_embs).to(device)

In [12]:
dpt_embs.shape

torch.Size([1, 1000, 147456])

In [4]:
for s in tqdm(range(dpt_embs.shape[1])):
    hidden_states = [dpt_embs[0,s,:].reshape(64,256,3,3)]
    with torch.no_grad():
        # hidden_states = model.neck(hidden_states)
        predicted_depth = model.head(hidden_states)

    # Make depth Image for visual inspection
    prediction = torch.nn.functional.interpolate(
        predicted_depth.unsqueeze(1),
        size=imsize,
        mode="bicubic",
        align_corners=False,
    )

    # visualize the prediction
    # output = prediction.squeeze().cpu().numpy()
    # formatted = (output * 255 / np.max(output)).astype("uint8")
    # depth = Image.fromarray(formatted)

    # Make latent reps for SD2
    cc = torch.nn.functional.interpolate(
        predicted_depth.unsqueeze(1),
        size=latentsize,
        mode="bicubic",
        align_corners=False,
    )
    depth_min, depth_max = torch.amin(cc, dim=[1, 2, 3], keepdim=True), torch.amax(cc, dim=[1, 2, 3],
                                                                                    keepdim=True)
    cc = 2. * (cc - depth_min) / (depth_max - depth_min) - 1.

    np.save(f'{savedir}/{s:06}.npy',cc.to('cpu').detach().numpy())
    # depth.save(f'{savedir_img}/{s:06}.png')

NameError: name 'dpt_embs' is not defined